# Mnist classification with NNs
A first example of a simple Neural Network, applied to a well known dataset.

In [1]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Model
from tensorflow.keras import utils
import numpy as np

2024-02-21 16:20:31.466801: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Let us load the mnist dataset

In [2]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 8s 1us/step


In [3]:
print(x_train.shape)
print("pixel range is [{},{}]".format(np.min(x_train),np.max(x_train)))

(60000, 28, 28)
pixel range is [0,255]


We normalize the input in the range [0,1]

In [4]:
x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.

x_train = np.reshape(x_train,(60000,28*28))
x_test = np.reshape(x_test,(10000,28*28))

The output of the network will be a proability distribution over the different categories. Similarly, we generate a ground truth distribution, and the training objective will consist in minimizing their distance (categorical crossentropy). The ground truth distribution is the so called "categorical" distribution: if x has label l, the corresponding categorical distribution has probaility 1 for the category l, and 0 for all the others.

In [5]:
print(y_train[0])
y_train_cat = utils.to_categorical(y_train)
print(y_train_cat[0])
y_test_cat = utils.to_categorical(y_test)

5
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]


Our first Netwok just implements logistic regression

In [6]:
xin = Input(shape=(28*28))
res = Dense(10,activation='softmax')(xin)

mynet = Model(inputs=xin,outputs=res)

In [7]:
mynet.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense (Dense)               (None, 10)                7850      
                                                                 
Total params: 7850 (30.66 KB)
Trainable params: 7850 (30.66 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Now we need to compile the network.
In order to do it, we need to pass two mandatory arguments:


*   the **optimizer**, in charge of governing the details of the backpropagation algorithm
*   the **loss function**

Several predefined optimizers exist, and you should just choose your favourite one. A common choice is Adam, implementing an adaptive lerning rate, with momentum

Optionally, we can specify additional metrics, mostly meant for monitoring the training process.

In [8]:
mynet.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

Finally, we fit the model over the trianing set. 

Fitting, just requires two arguments: training data e ground truth, that is x and y. Additionally we can specify epochs, batch_size, and many additional arguments.

In particular, passing validation data allow the training procedure to measure loss and metrics on the validation set at the end of each epoch.

In [9]:
mynet.fit(x_train,y_train_cat, shuffle=True, epochs=10, batch_size=32,validation_data=(x_test,y_test_cat))

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.4699 - accuracy: 0.8770 - val_loss: 0.3060 - val_accuracy: 0.9162
Epoch 2/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3041 - accuracy: 0.9153 - val_loss: 0.2800 - val_accuracy: 0.9219
Epoch 3/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2829 - accuracy: 0.9205 - val_loss: 0.2727 - val_accuracy: 0.9237
Epoch 4/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2738 - accuracy: 0.9239 - val_loss: 0.2674 - val_accuracy: 0.9249
Epoch 5/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2664 - accuracy: 0.9255 - val_loss: 0.2632 - val_accuracy: 0.9274
Epoch 6/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2618 - accuracy: 0.9272 - val_loss: 0.2648 - val_accuracy: 0.9266
Epoch 7/10
1875/1875 [==============================] - 3s 2ms/step - loss: 0.2586 - accuracy: 0.9282 - val_loss: 0.2651 - val_accuracy:

In [12]:
xin = Input(shape=(28*28))
x = Dense(50,activation='relu')(xin) # add 1 layer between input and output
res = Dense(10,activation='softmax')(x)

mynet2 = Model(inputs=xin,outputs=res)

In [13]:
mynet2.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 784)]             0         
                                                                 
 dense_3 (Dense)             (None, 50)                39250     
                                                                 
 dense_4 (Dense)             (None, 10)                510       
                                                                 
Total params: 39760 (155.31 KB)
Trainable params: 39760 (155.31 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Layer (type)                ;Output Shape              ;Param #   
-=================================================================

 input_3 (InputLayer)        ;[(None, 784)]             ;0         
                                                                 
 dense_3 (Dense)             ;(None, 50)                ;39250     (this is 784*50 + 50)
                                                                 
 dense_4 (Dense)             ;(None, 10)                ;510       (this is 50*10 + 10)
                                                                 
So we can generalize that the parameters of each layer are `previous_layer_shape * present_layer_shape + present_layer_shape`

In [14]:
mynet2.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [15]:
mynet2.fit(x_train,y_train_cat, shuffle=True, epochs=10, batch_size=32,validation_data=(x_test,y_test_cat))

Epoch 1/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.3232 - accuracy: 0.9096 - val_loss: 0.1804 - val_accuracy: 0.9460
Epoch 2/10
1875/1875 [==============================] - 4s 2ms/step - loss: 0.1593 - accuracy: 0.9527 - val_loss: 0.1374 - val_accuracy: 0.9575
Epoch 3/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.1179 - accuracy: 0.9653 - val_loss: 0.1191 - val_accuracy: 0.9637
Epoch 4/10
1875/1875 [==============================] - 5s 2ms/step - loss: 0.0943 - accuracy: 0.9718 - val_loss: 0.1069 - val_accuracy: 0.9679
Epoch 5/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0796 - accuracy: 0.9751 - val_loss: 0.1004 - val_accuracy: 0.9683
Epoch 6/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0686 - accuracy: 0.9789 - val_loss: 0.1028 - val_accuracy: 0.9703
Epoch 7/10
1875/1875 [==============================] - 5s 3ms/step - loss: 0.0596 - accuracy: 0.9813 - val_loss: 0.0929 - val_accuracy:

An amazing improvement. WOW!

# Exercises

1.   Add additional Dense layers and check the performance of the network
2.   Replace 'relu' with different activation functions
3. Adapt the network to work with the so called sparse_categorical_crossentropy
4. the fit function return a history of training, with temporal sequences for all different metrics. Make a plot.

